Installing and importing

In [2]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn

     -------------------------------------- 455.9/455.9 MB 1.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [7]:
df=pd.read_csv('Data/train.csv')

In [8]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [11]:
df.iloc[69]['comment_text']

'"\n\nParzival418, you are trying to scare ThAtSo. How can you say that the comments of ThAtSo are ""insulting""? ThAtSo is trying to suggest that there should be no original research in the Wikipedia. And, he is right. "'

In [16]:
df[df['toxic']==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


Preprocessing data

In [18]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      1.0.0
addict                       2.4.0
aiofiles                     0.8.0
anyio                        3.5.0
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asgiref                      3.5.0
asttokens                    2.0.5
astunparse                   1.6.3
attrs                        21.4.0
autopep8                     1.7.0
backcall                     0.2.0
beautifulsoup4               4.10.0
bleach                       4.1.0
bokeh                        2.4.2
cachetools                   5.2.0
certifi                      2021.10.8
cffi                         1.15.0
charset-normalizer           2.0.12
click                        8.1.2
colorama                     0.4.4
commonmark                   0.9.1
cvzone                       1.5.6
cycler                       0.11.0
debugpy                      1.6.0
decorator                    5.1.1
def


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from tensorflow.keras.layers import TextVectorization

In [21]:
TextVectorization??

In [31]:
X=df['comment_text']
y=df[df.columns[2:]].values

In [32]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [33]:
MAX_FEATURES = 200000

In [34]:
vectorizer=TextVectorization(max_tokens=MAX_FEATURES,
                            output_sequence_length=1800,
                            output_mode='int')

In [36]:
vectorizer.adapt(X.values)

In [46]:
vectorized_text=vectorizer(X.values)

In [47]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32141,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [51]:
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8) #help prevent bottelnecks

In [53]:
batch_x, batch_y=dataset.as_numpy_iterator().next()

In [54]:
batch_x.shape

(16, 1800)

In [55]:
batch_y.shape

(16, 6)

In [59]:
int(len(dataset)*.7)

6981

In [56]:
train=dataset.take(int(len(dataset)*.7))
val=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))#validation partition
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [62]:
len(test)

997

In [63]:
train_generator=train.as_numpy_iterator()

In [64]:
train_generator.next()

(array([[ 2395,   305,     2, ...,     0,     0,     0],
        [23545, 15361,    82, ...,     0,     0,     0],
        [ 6839, 11188,  5424, ...,     0,     0,     0],
        ...,
        [  175,    40,     8, ...,     0,     0,     0],
        [  908,    11, 16449, ...,     0,     0,     0],
        [  340,   340,     0, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

Building the deep neural network

Create Sequential model

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [67]:
model=Sequential()

In [70]:
model.add(Embedding(MAX_FEATURES+1,32))#An extra one is to embedd the words that were beyond the range
#LSTM Layer activation is tanh beacause tanh is required for gpu computation of LSTM

#Bidirectional is required because hate is considered as toxic but like dont't hate is non-toxic so bidirectional is required to handle these kind of cases
model.add(Bidirectional(LSTM(32, activation='tanh'))) 
#3 dense layer for feautre extraction with an activation of relu
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
#Final layer maps to the number of different outputs we have got using sigmoid activation it makes our output to be between 0 to 1
model.add(Dense(6, activation='sigmoid'))

In [71]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [72]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [73]:
history=model.fit(train,epochs=1,validation_data=val)

6981/6981 [==============================] - 1995s 285ms/step - loss: 0.1436 - val_loss: 0.1414
